In [1]:
pip install --upgrade together

Note: you may need to restart the kernel to use updated packages.Collecting together
     ---------------------------------------- 43.8/43.8 kB ? eta 0:00:00
     -------------------------------------- 395.2/395.2 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
     -------------------------------------- 365.2/365.2 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 50.4/50.4 kB ? eta 0:00:00
     ---------------------------------------- 76.4/76.4 kB ? eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.8.10
    Uninstalling tabulate-0.8.10:
      Successfully uninstalled tabulate-0.8.10
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
ydata-profiling 4.6.2 requires matplotlib<=3.7.3,>=3.2, but you have matplotlib 3.8.2 which is incompatible.
ydata-profiling 4.6.2 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.2 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


In [5]:
df_tuning_w_output = pd.read_csv('tuning_data_w_output_2.csv')

In [6]:
df_tuning_w_output

,Comments,Output
0,Christian was fabulous. We would highly recomm...,Ambiance - Great||Menu - Elevated||Price - Rea...
1,As soon as we walked in it was very very warm ...,Food - Soup dumplings (delicate and flavorful)...
2,Visited from Hawaii and was looking for a plac...,Food - Amazing||Music - Live performance||Serv...
3,1618 Asian fusion had great service! Shout out...,"Food - Bao buns (delicious), General Tso chick..."
4,Huge menu. Egg rolls for appetizer and Shrimp...,"Food - Egg rolls (nice fry), Shrimp Panang Cur..."
...,...,...
194,Skip the Zha Jiang Noodles (Dan Dan Noodles)\n...,"Food - soup dumplings (recommended), salt and ..."
195,Fusion restaurant located in the west downtown...,"Food - gator with Sichuan seasoning (liked, nu..."
196,"I think we really like this place, it is close...",Price - high
197,Cocktails and food were amazing and the restau...,"Food - truffle egg fried rice (favorite), pork..."


In [7]:
import together

In [106]:
together.api_key = "API"

In [170]:
# lets use our base model to see how it works before we finetune it

base_model_name = "togethercomputer/Llama-2-7B-32K-Instruct"

In [171]:
test_chat_prompt = """"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp Panang Curry (started great but got sweeter, needed some acid to balance sweetness), Vegetable curry (started great but got sweeter, needed some acid to balance sweetness)||Menu - Huge", Example 2 - "Service - Great||Food - Chicken Tikka Masala (nice and spicy), Lamb Biryani (aromatic and flavorful)||Drinks - Mango Lassi (refreshing), Masala Chai (aromatic)", Example 3 - "Food - Sushi (fresh and delicious), Ramen (flavorful broth), Tempura (crispy)||Ambiance - Cozy and inviting||Service - Friendly and attentive". 
<</SYS>> Below is the review on which you need to perform the task:\n\nOh my! Where do I start? Drinks? Amazing. Food...we totally overate because it was all soooo delicious. We sampled the dim sum and appetizers as well as the drink menu. Nothing disappointed us! Our waiter, Alessio, was fun, knowledgeable and made great recommendations. Overall I give this place 5 stars from start to finish. We will be back to try more of the delicious menu! Also, get the soup dumplings!!! Both of them. If you enjoy seafood, the crab soup dumplings (not pictured because I gobbled them up) are absolutely decadent. Yum![/INST]"""
test_chat_prompt

'"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp P

In [202]:
print(base_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = base_model_name,
  max_tokens = 256,
  temperature = 0.7,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>', '[/INST]']
)

# print generated text
print(output['prompt'][0]+" ->\n"+output['output']['choices'][0]['text'])

togethercomputer/Llama-2-7B-32K-Instruct
"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 

In [175]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):
    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt

format_to_llama2_chat(
    "You are a good robot",
    [("hi robot", "hello human"),("are you good?", "yes im good"),("are you bad?", "no, im good")]
)

'<s>[INST] <<SYS>> You are a good robot <</SYS>> hi robot [/INST] hello human </s> are you good? [/INST] yes im good </s> are you bad? [/INST] no, im good </s>'

In [176]:
data_list = []

system_prompt = """I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp Panang Curry (started great but got sweeter, needed some acid to balance sweetness), Vegetable curry (started great but got sweeter, needed some acid to balance sweetness)||Menu - Huge", Example 2 - "Service - Great||Food - Chicken Tikka Masala (nice and spicy), Lamb Biryani (aromatic and flavorful)||Drinks - Mango Lassi (refreshing), Masala Chai (aromatic)", Example 3 - "Food - Sushi (fresh and delicious), Ramen (flavorful broth), Tempura (crispy)||Ambiance - Cozy and inviting||Service - Friendly and attentive"."""

for index, row in df_tuning_w_output.iterrows():

    instruction_input_separator = "\n\n"
    comment = row['Comments']
    output = row['Output']
    instruction = "Below is the review on which you need to perform the task:"

    training_sequence = format_to_llama2_chat(
        system_prompt,
        [(instruction+instruction_input_separator+comment,output)]
    )

    data_list.append({
        "text":training_sequence
    })

print(len(data_list))
print(data_list[0])

199
{'text': '<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fr

In [177]:
# save the reformatted dataset locally
together.Files.save_jsonl(data_list, "Reviews_tuning_2.jsonl")

Wrote 199 records to Reviews_tuning_2.jsonl


In [178]:
# check your data with your base model prompting type before uploading
resp = together.Files.check(file="Reviews_tuning_2.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.0 GB', 'num_samples': 199}


In [116]:
resp = together.Files.list()
print(resp)

{'data': [], 'object': 'list'}


In [179]:
# upload your dataset file to together and save the file-id, youll need it to start your finetuning run
file_resp = together.Files.upload(file="Reviews_tuning_2.jsonl")
file_id = file_resp["id"]
print(file_resp)

Uploading Reviews_tuning_2.jsonl: 100%|██████████████████████████████████████████████| 484k/484k [00:01<00:00, 398kB/s]

{'filename': 'Reviews_tuning_2.jsonl', 'id': 'file-2d2d3eba-f802-4e83-b1fa-0039d3884fee', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.0 GB', 'num_samples': 199}}


In [180]:
# Submit your finetune job
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 8,
  batch_size = 1,
  n_checkpoints = 8,
  learning_rate = 5e-5,
  wandb_api_key = 'API',
  #estimate_price = True,
  suffix = 'reviews',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

{'training_file': 'file-2d2d3eba-f802-4e83-b1fa-0039d3884fee', 'validation_file': '', 'model_output_name': 'vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13', 'model_output_path': 's3://together-dev/finetune/65f2829777703cac1fac8bcd/vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13/ft-448a87e1-f583-4e97-9bab-e177454b1a94', 'Suffix': 'reviews', 'model': 'togethercomputer/Llama-2-7B-32K-Instruct', 'n_epochs': 8, 'n_checkpoints': 8, 'batch_size': 4, 'learning_rate': 5e-05, 'eval_steps': 0, 'user_id': '65f2829777703cac1fac8bcd', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-03-14T18:06:13.23Z', 'updated_at': '2024-03-14T18:06:13.23Z', 'status': 'pending', 'owner_address': '0x8f0b05017314be03488cf0a6a9e9cf789910fc3b', 'id': 'ft-448a87e1-f583-4e97-9bab-e177454b1a94', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_pri

In [184]:
# run this from time to time to check on the status of your job
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

{'training_file': 'file-2d2d3eba-f802-4e83-b1fa-0039d3884fee', 'validation_file': '', 'model_output_name': 'vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13', 'model_output_path': 's3://together-dev/finetune/65f2829777703cac1fac8bcd/vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13/ft-448a87e1-f583-4e97-9bab-e177454b1a94-2024-03-14-11-25-32', 'Suffix': 'reviews', 'model': 'togethercomputer/Llama-2-7B-32K-Instruct', 'n_epochs': 8, 'n_checkpoints': 8, 'batch_size': 4, 'learning_rate': 5e-05, 'eval_steps': 0, 'user_id': '65f2829777703cac1fac8bcd', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-03-14T18:06:13.23Z', 'updated_at': '2024-03-14T18:31:09.18Z', 'status': 'completed', 'owner_address': '0x8f0b05017314be03488cf0a6a9e9cf789910fc3b', 'id': 'ft-448a87e1-f583-4e97-9bab-e177454b1a94', 'job_id': '16509', 'token_count': 1317

completed
True
[{'object': 'fine-tune-event', 'created_at': '2024-03-14T18:17:44Z', 'level': 'Info', 'message': 'Uploaded checkpoint, at step 2', 'type': 'CHECKPOINT_SAVE', 'param_count': 0, 'token_count': 0, 'wandb_url': '', 'checkpoint_path': 's3://together-dev/finetune/65f2829777703cac1fac8bcd/vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13/ft-448a87e1-f583-4e97-9bab-e177454b1a94/checkpoint_2.tar.zst', 'model_path': '', 'training_offset': 0, 'hash': '9185674569769216317'}, {'object': 'fine-tune-event', 'created_at': '2024-03-14T18:20:41Z', 'level': 'Info', 'message': 'Uploaded checkpoint, at step 4', 'type': 'CHECKPOINT_SAVE', 'param_count': 0, 'token_count': 0, 'wandb_url': '', 'checkpoint_path': 's3://together-dev/finetune/65f2829777703cac1fac8bcd/vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13/ft-448a87e1-f583-4e97-9bab-e177454b1a94/checkpoint_4.tar.zst', 'model_path': '', 'training_offset': 0, 'hash': '-1175168456510771533'}, {'obj

In [185]:
# replace this name with the name of your newly finetuned model
new_model_name = 'vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

166 models available


True

In [203]:
# deploy your newly finetuned model
together.Models.start(new_model_name)

{'success': True,
 'value': '4e7b77772775129d5da507bf76d615601991df273e66511a35bb20aac1b15924-d820c5dfebf627348fb6959dfbf77ec4683f3c87b7cbbe41568abd1e6e70e54f'}

In [205]:
# check if your model is finished deploying, if this returns {"ready": true}, you model is ready for inference
together.Models.ready(new_model_name)

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '65f3426d5b280f2af1ad4529',
  'name': 'vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13',
  'display_type': 'chat',
  'description': "Extending LLaMA-2 to 32K context, built with Meta's Position Interpolation and Together AI's data recipe and system optimizations, instruction tuned by Together",
  'creator_organization': 'vgupta701@gmail.com',
  'hardware_label': 'L40',
  'num_parameters': 7000000000,
  'release_date': '2024-03-14T18:31:09.205Z',
  'show_in_playground': True,
  'owner': 'vgupta701@gmail.com',
  'owner_address': '0x8f0b05017314be03488cf0a6a9e9cf789910fc3b',
  'owner_userid': '65f2829777703cac1fac8bcd',
  'parent': 'togethercomputer/Llama-2-7B-32K-Instruct',
  'base': 'togethercomputer/Llama-2-7B-32K-Instruct',
  'path': 'r2://together-dev/finetune/65f2829777703cac1fac8bcd/vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13/ft-448a87e1-f583-4e97-9bab-e177454b1a94-2

In [206]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  #prompt = """"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp Panang Curry (started great but got sweeter, needed some acid to balance sweetness), Vegetable curry (started great but got sweeter, needed some acid to balance sweetness)||Menu - Huge", Example 2 - "Service - Great||Food - Chicken Tikka Masala (nice and spicy), Lamb Biryani (aromatic and flavorful)||Drinks - Mango Lassi (refreshing), Masala Chai (aromatic)", Example 3 - "Food - Sushi (fresh and delicious), Ramen (flavorful broth), Tempura (crispy)||Ambiance - Cozy and inviting||Service - Friendly and attentive". 
#<</SYS>> Had dinner at Qi the other night and food was delicious! We started with the recommended Shanghai soup dumplings (5 pieces) and beef Bao (3 baos) and both were very tasty. We then split the truffle egg fried rice and spicy chicken with pine nuts (pictured) which were great as well! Those 4 items definitely filled up my husband and I. Ambience is good, decor very beautiful, and they are doing an okay job at spacing out guests. We specially reserved a table outdoors hoping to be spaced out more, however back to back tables were almost touching so not the most social distanced for our liking. All staff is wearing masks, some wearing gloves. No temperature checks.[/INST]""",
  model = new_model_name,
  max_tokens = 200,
  temperature = 0.7,
  top_k = 50,
  top_p = 0.7,
  repetition_penalty = 1.1,
  stop = ['</s>', '[/INST]','\n']
)

# print generated text
print(output['prompt'][0]+" ->\n\n"+output['output']['choices'][0]['text'])

vgupta701@gmail.com/Llama-2-7B-32K-Instruct-reviews-2024-03-14-18-06-13
"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the f

In [207]:
# print generated text
print(output['prompt'][0]+" ->\n\n"+"Food - Crab Soup Dumplings (decadent), Dim Sum (delicious), Appetizers (delicious)||Drinks - Amazing||Service - Fun, Knowledgeable, Great recommendations")

"<s>[INST] <<SYS>> I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp Pa

In [195]:
Full_Data = pd.read_csv('1618_Qi.CSV')
Full_Data = Full_Data.drop(columns = 'Unnamed: 0')
Full_Data

,Restaurant,Comments
0,1618 Asian Fusion,Christian was fabulous. We would highly recomm...
1,1618 Asian Fusion,As soon as we walked in it was very very warm ...
2,1618 Asian Fusion,Visited from Hawaii and was looking for a plac...
3,1618 Asian Fusion,1618 Asian fusion had great service! Shout out...
4,1618 Asian Fusion,Huge menu. Egg rolls for appetizer and Shrimp...
...,...,...
1038,Qi Austin,Qi is a new restaurant in downtown Austin. Gre...
1039,Qi Austin,Very underwhelmed by the flavor and seasoning ...
1040,Qi Austin,My sister and I had a fantastic brunch at Qi t...
1041,Qi Austin,We spent a lot of money here and it was well w...


In [196]:
Full_Data['Output'] = ""
Full_Data

,Restaurant,Comments,Output
0,1618 Asian Fusion,Christian was fabulous. We would highly recomm...,
1,1618 Asian Fusion,As soon as we walked in it was very very warm ...,
2,1618 Asian Fusion,Visited from Hawaii and was looking for a plac...,
3,1618 Asian Fusion,1618 Asian fusion had great service! Shout out...,
4,1618 Asian Fusion,Huge menu. Egg rolls for appetizer and Shrimp...,
...,...,...,...
1038,Qi Austin,Qi is a new restaurant in downtown Austin. Gre...,
1039,Qi Austin,Very underwhelmed by the flavor and seasoning ...,
1040,Qi Austin,My sister and I had a fantastic brunch at Qi t...,
1041,Qi Austin,We spent a lot of money here and it was well w...,


In [197]:
import time

system_prompt = """I have a review for a restaurant. You are a restaurant review summarizer. I want you to produce an output that would contain the key topics discussed by the customer in the review and a brief takeaway for each topic. Topics should be within - Food, Drinks, Ambiance, Price, Service, Parking, Menu, Location and Music. Please include one topic for sure called Food for which the takeaway would simply be all the food items mentioned by the customer and a short assessment of customer’s view about that food in brackets not going beyond 5 words in length. If no food item is mentioned and the word food is missing, skip that topic. If no food item is mentioned but the word food is mentioned, summarize customer’s opinion of the food in general. Please separate any two topics by || and any two takeaways by a comma. Please stick to this format and do not output any extra text. Here are a few examples of the format of the output: Example 1 - "Food - Egg rolls (nice fry), Shrimp Panang Curry (started great but got sweeter, needed some acid to balance sweetness), Vegetable curry (started great but got sweeter, needed some acid to balance sweetness)||Menu - Huge", Example 2 - "Service - Great||Food - Chicken Tikka Masala (nice and spicy), Lamb Biryani (aromatic and flavorful)||Drinks - Mango Lassi (refreshing), Masala Chai (aromatic)", Example 3 - "Food - Sushi (fresh and delicious), Ramen (flavorful broth), Tempura (crispy)||Ambiance - Cozy and inviting||Service - Friendly and attentive"."""
instruction = "Below is the review on which you need to perform the task:\n\n"

for index, row in Full_Data.iterrows():
    print(index)
    comment = row['Comments'] 
    prompt_sequence = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>> {instruction}\n\n{comment} [/INST]"""
    output = together.Complete.create(
      prompt = prompt_sequence,
      model = new_model_name,
      max_tokens = 200,
      temperature = 0.7,
      top_k = 50,
      top_p = 0.7,
      repetition_penalty = 1.1,
      stop = ['</s>', '[/INST]','\n']
    )
    row['Output']  = output['output']['choices'][0]['text']
    time.sleep(1)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [199]:
index = 102

print(Full_Data['Comments'][index])

print("*************************************")

print(Full_Data['Output'][index])

The 1618 egg rolls matched with the Xiao Long soup dumplings is perfection. Thank you to Abigail our server and bartender who made our experience wonderful! Overall, 1618 is our go to for date night because food and service is always on point.
*************************************
Food - Egg rolls (matched well with Xiao Long soup dumplings), Soup Dumplings (perfect)||Service - Wonderful 


In [90]:
Full_Data['Output']

0       Topics discussed by the customer in the review...
1       Topics discussed by the customer in the review...
2       Topics discussed by the customer in the review...
3       Topics discussed by the customer in the review...
4       Topics discussed by the customer in the review...
                              ...                        
1038                                                     
1039                                                     
1040                                                     
1041                                                     
1042                                                     
Name: Output, Length: 1043, dtype: object

In [201]:
Full_Data.to_csv('Full_Data_2.csv', index=False)

In [200]:
# stop your model and you will no longer be paying for it
together.Models.stop(new_model_name)

{'success': True}